In [1]:
import streamlit as st
import plotly.graph_objects as go
import pandas as pd

# Sample data
top = pd.DataFrame({'UserName': ['A', 'B', 'C', 'D'], 'Nom': [50, 30, 15, 5]})

fig = go.Figure()

top['Percent'] = top['Nom'] / top['Nom'].sum() * 100

# Create formatted labels with percentages
top['LabelWithPercent'] = top['UserName'] + ' (' + top['Percent'].round(2).astype(str) + '%)'


fig.add_trace(go.Pie(
    labels=top['LabelWithPercent'],
    values=top['Nom'],
    pull=[0.1 if i == 1 else 0 for i in range(len(top))],
    marker=dict(colors=['#636EFA', '#EF553B', '#00CC96', '#AB63FA'], line=dict(color='#FFFFFF', width=2)),
    textinfo='value', 
    textposition='inside',
    textfont=dict(color='#FFFFFF', size=12),
    hoverinfo='label+percent+value',
    # Display percentages in the legend
    legendgroup='legend',
    showlegend=True,
    sort=False,
))

# Update layout to include custom legend with percentages
fig.update_layout(
    legend=dict(
        title="Legend",
        itemsizing='constant',
        font=dict(size=10)
    ),
    annotations=[dict(text='UserName', x=0.5, y=0.5, font_size=20, showarrow=False)]
)

# Show the chart in Streamlit
fig


In [1]:
import streamlit as st
import pydeck as pdk
import pandas as pd

# Sample data for the map
DATA_URL = "https://raw.githubusercontent.com/uber-common/deck.gl-data/master/website/bart-stations.json"

# Load data
@st.cache_data
def load_data():
    data = pd.read_json(DATA_URL)
    return data

data = load_data()

# Define a layer to display on a map
layer = pdk.Layer(
    "ScatterplotLayer",
    data,
    pickable=True,
    opacity=0.8,
    stroked=True,
    filled=True,
    radius_scale=10,
    radius_min_pixels=5,
    radius_max_pixels=100,
    line_width_min_pixels=1,
    get_position="coordinates",
    get_radius=100,
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

# Set the viewport location
view_state = pdk.ViewState(
    latitude=37.7749,
    longitude=-122.4194,
    zoom=10,
    pitch=50,
)

# Render the deck.gl map
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{name}"})

st.pydeck_chart(r)


2024-08-06 10:50:04.523 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-08-06 10:50:04.595 
  command:

    streamlit run /Users/obertys/miniforge3/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-08-06 10:50:04.596 No runtime found, using MemoryCacheStorageManager


DeltaGenerator()

In [28]:
import pyodbc
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta

# Define the connection parameters
server = 'marlodj-ecobank-db-server.database.windows.net'
database = 'MarlodjCore'
username = 'marlodj-admin'
password = 'tR0i48L658jQ'
driver = '{ODBC Driver 17 for SQL Server}'
#MARS_Connection='yes'
# Create the connection string
connection_string = (
    f'DRIVER={driver};'
    f'SERVER={server};'
    f'PORT=1433;'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    f'PWD={password};'
    # f'TRUSTED_CONNECTION=yes;'
)



# Establish the connection
connection = pyodbc.connect(connection_string)
import pyodbc
from datetime import datetime, timedelta

def get_previous_workdays(date, days_back):
    workdays = []
    # Si la date actuelle est un jour ouvrable, l'ajouter à la liste
    if date.weekday() < 5:
        workdays.append(date.strftime('%Y-%m-%d'))
    # Continuer à ajouter les jours ouvrables précédents
    while len(workdays) < days_back:
        date -= timedelta(days=1)
        if date.weekday() < 5:  # 0-4 correspond à lundi-vendredi
            workdays.append(date.strftime('%Y-%m-%d'))
    return workdays

# Fonction pour sélectionner les données
# def view_all_data(days_back):
#     today = datetime.now()
#     workdays = get_previous_workdays(today, days_back)

#     # Si la date actuelle est un jour ouvrable, l'ajouter à la liste workdays
    
#     # Convertir la liste workdays en tuple pour les paramètres de la requête SQL
#     placeholders = tuple(workdays)
#     nbs_params=','.join('?' for _ in range(len(workdays)))
#     sql = f"""select u.FirstName,u.LastName,u.UserName, q.Date_Reservation,q.Date_Appel,TempAttenteMoyen,e.Nom, DATEDIFF(second,q.Date_Reservation,q.Date_Appel) as TempsAttenteReel,q.Date_Fin ,DATEDIFF(second,q.Date_Appel,q.Date_Fin) as TempOperation,s.NomService from reservation r ,[User] u,Etat e, queue q,Service s where r.id= q.reservationId and r.ServiceId=s.Id and u.Id=q.userId and e.Id=q.EtatId and 
#         CAST(q.Date_Reservation AS DATE) IN ({nbs_params}) ORDER BY q.Date_Reservation DESC
#     """
#     df = pd.read_sql_query(sql, connection,params=placeholders)
    
#     return df

def view_all_data(start_date, end_date):

    sql = f"""select u.FirstName,u.LastName,u.UserName, q.Date_Reservation,q.Date_Appel,TempAttenteMoyen, DATEDIFF(second,q.Date_Reservation,q.Date_Appel) as TempsAttenteReel,q.Date_Fin ,DATEDIFF(second,q.Date_Appel,q.Date_Fin) as TempOperation,e.Nom
,s.NomService,r.TypeOperationId,r.AgenceId ,a.NomAgence,a.Capacites from reservation r ,[User] u,Etat e, queue q,Service s,Agence a where r.id= q.reservationId and r.ServiceId=s.Id and u.Id=q.userId and e.Id=q.EtatId and 
        a.Id=r.AgenceId and CAST(q.Date_Reservation AS DATE) BETWEEN CAST(? AS datetime) AND CAST(? AS datetime) ORDER BY q.Date_Reservation DESC
    """
    df = pd.read_sql_query(sql, connection,params=(start_date, end_date), index_col=None)
    
    return df

def view_queue(start_date, end_date):
   
    sql = f"""select q.Date_Reservation,q.Date_Appel,TempAttenteMoyen, DATEDIFF(second,q.Date_Reservation,q.Date_Appel) as TempsAttenteReel,q.Date_Fin ,DATEDIFF(second,q.Date_Appel,q.Date_Fin) as TempOperation,e.Nom
,s.NomService,r.TypeOperationId,r.AgenceId ,a.NomAgence,a.Capacites from reservation r , queue q ,Service s ,Agence a,Etat e where r.id= q.reservationId and r.ServiceId=s.Id and e.Id=q.EtatId
        and a.Id=r.AgenceId and CAST(q.Date_Reservation AS DATE) BETWEEN CAST(? AS datetime) AND CAST(? AS datetime) ORDER BY q.Date_Reservation DESC
    """
    df = pd.read_sql_query(sql, connection,params=(start_date, end_date), index_col=None)

    return df  

def get_sqlData():
    sql=f""" SELECT 
    u.FirstName,
    u.LastName,
    u.UserName,
    q.Date_Reservation,
    q.Date_Appel,
    q.TempAttenteMoyen,
    DATEDIFF(second, q.Date_Reservation, q.Date_Appel) as TempsAttenteReel,
    q.Date_Fin,
    DATEDIFF(second, q.Date_Appel, q.Date_Fin) as TempOperation,
    e.Nom ,
    s.NomService,
    t.Label as Type_Operation,
    r.AgenceId,
    a.NomAgence,
    a.Capacites,
    a.Longitude,
    a.Latitude
FROM 
    reservation r
    LEFT JOIN TypeOperation t ON t.Id=r.TypeOperationId
    LEFT JOIN queue q ON r.id = q.reservationId
    LEFT JOIN Service s ON r.ServiceId = s.Id
    LEFT JOIN [User] u ON u.Id = q.userId  -- Utilisation de LEFT JOIN pour inclure les valeurs nulles
    LEFT JOIN Etat e ON e.Id = q.EtatId
    LEFT JOIN Agence a ON a.Id = r.AgenceId
WHERE Date_Reservation is not NULL and CAST(q.Date_Reservation AS DATE) BETWEEN CAST('2024-06-07' AS datetime) AND CAST('2024-09-08' AS datetime) 
ORDER BY 
    q.Date_Reservation DESC;
  """
    df = pd.read_sql_query(sql, connection, index_col=None)
    df_all = df[df['UserName'].notna()].reset_index(drop=True)
    df_queue=df.copy()
    return  df_all,df_queue


In [29]:
data,_=get_sqlData()

/Users/obertys/miniforge3/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [30]:
df=data.copy()

In [32]:
def stacked_service(data,type:str,concern:str,titre="Nombre de type d'opération par Service"):
    """
    Default values of type:
    'TempsAttenteReel' and 'TempOperation'
    """
    df=data.copy()
    df=df.sample(n=min(5000, len(data)),replace=False)
    df[type] = df[type].dropna()
    
    df=df.groupby([f'{type}', f'{concern}']).size().reset_index(name='Count')
    
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X(f'{type}:O', title='Service'),
        y=alt.Y('Count:Q', title='Nombre par Categorie'),
        color=alt.Color(f'{concern}:N', title="Type d'Opération"),
        order=alt.Order(f'{concern}:N')  # Ensures the stacking order
    ).properties(
        width=1000,
        height=400,
        title=f"{titre}"
    )
    return chart

stacked_service(df,type='NomService',concern='Type_Operation')

alt.Chart(...)

In [33]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Sample DataFrame
data = {
    'Time_bins': ['2024-08-01', '2024-08-01', '2024-08-02', '2024-08-02', '2024-08-03', '2024-08-03'],
    'Categorie': ['0-5min', '5-10min', '0-5min', '>10min', '5-10min', '>10min'],
    'Counts': [10, 5, 8, 3, 12, 7]
}

df = pd.DataFrame(data)

# Pivot the DataFrame
df_pivot = df.pivot(index='Time_bins', columns='Categorie', values='Counts').fillna(0)

# Plot the stacked bar chart
fig = px.bar(df_pivot, 
             x=df_pivot.index, 
             y=df_pivot.columns,
             title="Stacked Bar Chart Example",
             labels={'value':'Counts', 'Time_bins':'Time Bins'},
             text_auto=True)

# Display in Streamlit
fig


In [8]:
import pandas as pd

# Assume 'df' is your original dataframe with multiple columns
def generate_agence(df,num_new_agencies):

    rows_per_agency = int(len(df)/num_new_agencies)

    new_agencies = [f'Agence_{i+1}' for i in range(num_new_agencies)]

    # Update the 'NomAgence' column with new agency names in blocks of 500 rows
    for i, agency in enumerate(new_agencies):
        start_index = i * rows_per_agency
        end_index = start_index + rows_per_agency
        df.loc[start_index:end_index, 'NomAgence'] = agency
    return df 


In [13]:
import numpy as np

In [120]:
df['TempAttente']=df['TempsAttenteReel'].apply(lambda x: np.round(np.mean(x)/60).astype(int))


In [14]:
df['Categorie']=df['TempsAttenteReel'].apply(lambda x: '0-5min' if 0<=np.round(np.mean(x)/60).astype(int)<=5 else '5-10min' if 5<=np.round(np.mean(x)/60).astype(int)<=10 else '>10min' )

In [17]:
df.groupby(['NomAgence', 'Categorie']).size().reset_index(name='Count')

,NomAgence,Categorie,Count
0,Agence Principale,0-5min,8139
1,Agence Principale,5-10min,1850
2,Agence Principale,>10min,1739
3,Agence Principale,NaN,117


In [15]:
import numpy as np
df=data.copy()

In [16]:
#df['TempOperation'] = df['TempOperation'].dropna()

df['Categorie'] = df['TempOperation'].apply(lambda x: 
    '0-5min' if pd.notnull(x) and 0 <= np.round(x/60).astype(int) <= 5 else 
    '5-10min' if pd.notnull(x) and 5 < np.round(x/60).astype(int) <= 10 else 
    '>10min' if pd.notnull(x) else 'NaN'
)

In [6]:
data

,FirstName,LastName,UserName,Date_Reservation,Date_Appel,TempAttenteMoyen,TempsAttenteReel,Date_Fin,TempOperation,Nom,NomService,Type_Operation,AgenceId,NomAgence,Capacites,Longitude,Latitude
0,SALIMATA EVE,CISSOKHO,SCISSOKHO,2024-08-16 16:01:16.890,2024-08-16 16:18:39.847,0.0,1043.0,NaT,NaN,En attente,Clientele,None,32,Agence Principale,30,-17.469949,14.70138
1,SALIMATA EVE,CISSOKHO,SCISSOKHO,2024-08-16 15:51:52.427,2024-08-16 16:15:01.707,0.0,1389.0,2024-08-16 16:18:39.843,218.0,Traitée,Clientele,None,32,Agence Principale,30,-17.469949,14.70138
2,SALIMATA EVE,CISSOKHO,SCISSOKHO,2024-08-16 15:40:33.110,2024-08-16 16:09:53.937,0.0,1760.0,2024-08-16 16:15:01.700,308.0,Traitée,Clientele,None,32,Agence Principale,30,-17.469949,14.70138
3,SALIMATA EVE,CISSOKHO,SCISSOKHO,2024-08-16 15:05:51.223,2024-08-16 16:09:00.723,0.0,3789.0,2024-08-16 16:09:53.933,53.0,Traitée,Clientele,None,32,Agence Principale,30,-17.469949,14.70138
4,SALIMATA EVE,CISSOKHO,SCISSOKHO,2024-08-16 14:44:14.333,2024-08-16 15:37:43.863,0.0,3209.0,2024-08-16 16:09:00.720,1877.0,Traitée,Clientele,None,32,Agence Principale,30,-17.469949,14.70138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11840,LEON MARCELIN,NDONG,LNDONG2,2024-06-07 07:32:27.427,2024-06-07 08:05:09.973,0.0,1962.0,2024-06-07 08:08:19.713,190.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138
11841,AMINATA,SANE,amisane,2024-06-07 07:32:15.263,2024-06-07 07:59:42.827,0.0,1647.0,2024-06-07 08:13:43.730,841.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138
11842,AMINATA,SANE,amisane,2024-06-07 07:32:07.150,2024-06-07 07:50:59.710,0.0,1132.0,2024-06-07 07:59:42.820,523.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138
11843,AMINATA,SANE,amisane,2024-06-07 07:31:42.620,2024-06-07 07:50:48.230,0.0,1146.0,2024-06-07 07:50:59.703,11.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138


In [27]:
import altair as alt
import pandas as pd
import streamlit as st

def stacked_chart(data,type,concern):
    """
    Default values of type:
    'TempsAttenteReel' and 'TempOperation'
    """
    df=data.copy()
    df=df.sample(5000)
    df[type] = df[type].dropna()
    df['Categorie'] = df[type].apply(lambda x: 
    '0-5min' if 0 <= np.round(x/60).astype(int) <= 5 else 
    '5-10min' if 5 < np.round(x/60).astype(int) <= 10 else 
    '>10min'
)
    df=df.groupby([f'{concern}', 'Categorie']).size().reset_index(name='Count')
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X(f'{concern}:O', title=f'{concern}'),
        y=alt.Y('Count:Q', title='Nombre par Categorie'),
        color=alt.Color('Categorie:N', title='Categories'),
        order=alt.Order('Categorie:N')  # Ensures the stacking order
    ).properties(
        width=600,
        height=400
    )
    return chart

stacked_chart(data,'TempOperation','NomAgence')

alt.Chart(...)

In [157]:
import altair as alt
import pandas as pd
import streamlit as st

import altair as alt
import pandas as pd
import streamlit as st

# Sample data
# data = pd.DataFrame({
#     'NomAgence': ['Agency1'] * 6,
#     'TempsAttente': [2, 7, 3, 11, 6, 8],
#     'Date_Appel': pd.date_range(start='2024-08-01', periods=6, freq='D'),
#     'Categorie': ['0-5min', '5-10min', '0-5min', '>10min', '5-10min', '5-10min'],
#     'Time_Bins': ['2024-08-01 to 2024-08-07'] * 6
# })


# Create the Stacked Bar Chart
chart = alt.Chart(df1).mark_bar().encode(
    x=alt.X('NomAgence:O', title='Time Bins'),
    y=alt.Y('TempOperation:Q', title='Number of Calls'),
    color=alt.Color('UserName:N', title='Category'),
    order=alt.Order('UserName:N')  # Ensures the stacking order
).properties(
    width=600,
    height=400
)

# Display in Streamlit
st.write("Stacked Bar Chart Example")
chart



alt.Chart(...)

In [193]:
df['NomAgence'].unique()

array(['Agence29', 'Agence12', 'Agence Principale', 'Agence35',
       'Agence32', 'Agence8', 'Agence34', 'Agence20', 'Agence31',
       'Agence21', 'Agence27', 'Agence17', 'Agence7', 'Agence11',
       'Agence26', 'Agence24', 'Agence6', 'Agence19', 'Agence15',
       'Agence2', 'Agence1', 'Agence23', 'Agence28', 'Agence22',
       'Agence10', 'Agence18', 'Agence13', 'Agence3', 'Agence14',
       'Agence25', 'Agence9', 'Agence16', 'Agence5', 'Agence0',
       'Agence30', 'Agence4', 'Agence33'], dtype=object)

In [171]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
import numpy as np


df=data.copy()

def assign_to_bin(date,bins):
    date = pd.Timestamp(date).normalize()  # Convert string date to Timestamp and normalize (ignore time)
    for start, end in bins:
        start_date = pd.Timestamp(start).normalize()
        end_date = pd.Timestamp(end).normalize()
        if start_date <= date <= end_date:
            return f"{start_date.date()} to {end_date.date()}"
    return None

# # Number of new agencies to create
# num_new_agencies = 36

# # Number of rows per agency
# rows_per_agency = 500

# # Generate new agency names
# new_agencies = [f'Agence_{i+1}' for i in range(num_new_agencies)]

# # Update the 'NomAgence' column with new agency names in blocks of 500 rows
# for i, agency in enumerate(new_agencies):
#     start_index = i * rows_per_agency
#     end_index = start_index + rows_per_agency
#     df.loc[start_index:end_index, 'NomAgence'] = agency
    
def get_time_bins(min_date, max_date, bin_type):
    start_date = min_date
    time_bins = []

    if bin_type == 'Mois':
        offset = pd.DateOffset(months=1)
    elif bin_type == 'Semaine':
        offset = pd.DateOffset(weeks=1)
    elif bin_type == 'Annee':
        offset = pd.DateOffset(years=1)
    else:
        raise ValueError("bin_type must be 'month', 'week', or 'year'")

    while start_date <= max_date:
        if bin_type == 'Semaine':
            end_date = start_date + pd.DateOffset(days=6)
        else:
            end_date = (start_date + offset) - pd.DateOffset(days=1)

        # Ensure the end date does not exceed the max_date
        if end_date > max_date:
            end_date = max_date

        time_bins.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))

        # Move to the next bin
        start_date = end_date + pd.DateOffset(days=1)

    return time_bins


df=df.dropna(subset=['Date_Fin'])
# Convert columns to datetime
df['Date_Reservation'] = pd.to_datetime(df['Date_Reservation'])
df['Date_Fin'] = pd.to_datetime(df['Date_Fin'])


# Calculate the difference between the min and max dates
min_date = df['Date_Reservation'].min()
max_date = df['Date_Reservation'].max()
date_diff = (max_date - min_date).days

# Define the Time_Bin intervals based on the date difference
if date_diff == 0:
    unit='Heure'
    # Extract hour from Date_Reservation for hourly binning
    df['Hour'] = df['Date_Fin'].dt.hour
    bins = [i for i in range(7, 19)]  # From 7 to 18 hours
    labels = ['7-8am', '8-9am', '9-10am', '10-11am', '11-12pm', '12-1pm', '1-2pm', '2-3pm', '3-4pm', '4-5pm', '5-6pm']
    df['Time_Bin'] = pd.cut(df['Hour'], bins=bins, labels=labels, right=False)

elif 1 <= date_diff <=7:
    unit='Jour'
    df['Time_Bin'] = df['Date_Fin'].dt.day
    complete_dates=range(min_date.day,max_date.day+1)
elif 7 < date_diff <=84:
    unit="Semaine"
    bins=get_time_bins(min_date,max_date,unit)
    df['Time_Bin'] = df['Date_Fin'].apply(lambda x:assign_to_bin(x,bins))
    
elif 84 < date_diff <=365:
    unit="Mois"
    bins=get_time_bins(min_date,max_date,unit)
    df['Time_Bin'] = df['Date_Fin'].apply(lambda x:assign_to_bin(x,bins))
else:
    unit='Annee'
    bins=get_time_bins(min_date,max_date,unit)
    df['Time_Bin'] = df['Date_Fin'].apply(lambda x:assign_to_bin(x,bins))

# Group by Nom_Agence and Time_Bin, and calculate the average TempAttente
grouped_data = df.groupby(['NomAgence', 'Time_Bin'])[['TempOperation']].agg(( lambda x: np.round(np.mean(x)/60).astype(int))).reset_index()

# Calculate area under the curve for each agency
area_data = grouped_data.groupby('NomAgence')['TempOperation'].sum().reset_index()
area_data = area_data.sort_values(by='TempOperation', ascending=False)

# Select the top 5 agencies with the largest area under the curve
top_5_agences = area_data.head(5)['NomAgence'].tolist()


# Create a DataFrame with all combinations of agencies and time bins
if unit=="Jour":
    all_combinations = pd.MultiIndex.from_product([top_5_agences, sorted(complete_dates)], names=['NomAgence', 'Time_Bin']).to_frame(index=False)
else:
    
    all_combinations = pd.MultiIndex.from_product([top_5_agences, sorted(df['Time_Bin'].dropna().unique())], names=['NomAgence', 'Time_Bin']).to_frame(index=False)

all_combinations = pd.merge(all_combinations, grouped_data, on=['NomAgence', 'Time_Bin'], how='left').fillna(0)


# Create a figure with go.Figure
fig = go.Figure()

# Add traces for each agency
for agence in top_5_agences:
    agency_data = all_combinations[all_combinations['NomAgence'] == agence]
    fig.add_trace(go.Scatter(
        x=agency_data['Time_Bin'],
        y=agency_data['TempOperation'],
        mode='lines+markers',
        fill='tozeroy',
        name=agence
    ))

# Update layout for better visualization
fig.update_layout(
    title='Courbes des 5 Agences les Plus Lentes par TempAttente',
    xaxis_title=f'Intervalle de Temps en {unit}',
    yaxis_title='TempAttente Moyenne (minutes)',
    template='plotly_dark',
    legend_title='Agence'
)
# Ajouter une ligne horizontale avec une couleur différente des courbes
fig.add_shape(
    type="line",
    x0=all_combinations['Time_Bin'].min(),  # Début de la ligne sur l'axe x
    x1=all_combinations['Time_Bin'].max(),  # Fin de la ligne sur l'axe x
    y0=5,  # Position de la ligne sur l'axe y
    y1=5,  # Même que y0 pour que la ligne soit horizontale
    line=dict(color="yellow", width=2, dash="dot")  # Couleur différente (ici, noir)
)

# Display the chart in Streamlit
fig


In [137]:
date_diff=367
['Semaine', 'Mois', 'Annee'][int(date_diff > 84) + int(date_diff > 365)]

'Annee'

In [129]:
df['Time_Bin'].value_counts()

10-11am    1443
3-4pm      1433
11-12pm    1417
9-10am     1285
2-3pm      1270
12-1pm     1250
8-9am      1070
4-5pm       980
1-2pm       973
7-8am       539
5-6pm       156
Name: Time_Bin, dtype: int64

In [161]:


df=data.copy()

# # Number of new agencies to create
# num_new_agencies = 36

# # Number of rows per agency
# rows_per_agency = 500

# # Generate new agency names
# new_agencies = [f'Agence_{i+1}' for i in range(num_new_agencies)]

# # Update the 'NomAgence' column with new agency names in blocks of 500 rows
# for i, agency in enumerate(new_agencies):
#     start_index = i * rows_per_agency
#     end_index = start_index + rows_per_agency
#     df.loc[start_index:end_index, 'NomAgence'] = agency
def assign_to_bin(date,bins):
    date = pd.Timestamp(date).normalize()  # Convert string date to Timestamp and normalize (ignore time)
    for start, end in bins:
        start_date = pd.Timestamp(start).normalize()
        end_date = pd.Timestamp(end).normalize()
        if start_date <= date <= end_date:
            return f"{start_date.date()} to {end_date.date()}"
    return None   
def get_time_bins(min_date, max_date, bin_type):
    start_date = min_date
    time_bins = []

    if bin_type == 'Mois':
        offset = pd.DateOffset(months=1)
    elif bin_type == 'Semaine':
        offset = pd.DateOffset(weeks=1)
    elif bin_type == 'Annee':
        offset = pd.DateOffset(years=1)
    else:
        raise ValueError("bin_type must be 'month', 'week', or 'year'")

    while start_date <= max_date:
        if bin_type == 'Semaine':
            end_date = start_date + pd.DateOffset(days=6)
        else:
            end_date = (start_date + offset) - pd.DateOffset(days=1)

        # Ensure the end date does not exceed the max_date
        if end_date > max_date:
            end_date = max_date

        time_bins.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))

        # Move to the next bin
        start_date = end_date + pd.DateOffset(days=1)

    return time_bins

def area_graph(df,concern='UserName',time='TempOperation',date_to_bin='Date_Fin',seuil=5,title='Courbe'):
    df=df.dropna(subset=[date_to_bin])

    # Convert columns to datetime
    df['Date_Reservation'] = pd.to_datetime(df['Date_Reservation'])
    df[date_to_bin] = pd.to_datetime(df[date_to_bin])


    # Calculate the difference between the min and max dates
    min_date = df['Date_Reservation'].min()
    max_date = df['Date_Reservation'].max()
    date_diff = (max_date - min_date).days

    # Define the Time_Bin intervals based on the date difference
    if date_diff == 0:
        unit, df['Time_Bin'] = 'Heure', pd.cut(df[date_to_bin].dt.hour, bins=range(7, 19), labels=[f'{i}-{i+1}am' for i in range(7, 12)] + [f'{i-12}-{i-11}pm' for i in range(12, 18)], right=False)
    elif 1 <= date_diff <=7:
        unit, df['Time_Bin'], complete_dates = 'Jour', df[date_to_bin].dt.day, range(min_date.day, max_date.day + 1)
    else:
        unit = ['Semaine', 'Mois', 'Annee'][int(date_diff > 84) + int(date_diff > 365)]
        bins = get_time_bins(min_date, max_date, unit)
        df['Time_Bin'] = df[date_to_bin].apply(lambda x: assign_to_bin(x, bins))


    # Group by Nom_Agence and Time_Bin, and calculate the average TempAttente
    grouped_data = df.groupby([concern, 'Time_Bin'])[[time]].agg(( lambda x: np.round(np.mean(x)/60).astype(int))).reset_index()

    # Select the top 5 agencies with the largest area under the curve
    if len(df['NomAgence'].unique())==1 and concern=='UserName':
        top_agences=grouped_data[concern].unique()
    else:
        top_agences =grouped_data.groupby(concern)[time].sum().nlargest(5).index.tolist()
    


    # Create a DataFrame with all combinations of agencies and time bins
    if unit=="Jour":
        all_combinations = pd.MultiIndex.from_product([top_agences, sorted(complete_dates)], names=[concern, 'Time_Bin']).to_frame(index=False)
    else:
        
        all_combinations = pd.MultiIndex.from_product([top_agences, sorted(df['Time_Bin'].dropna().unique())], names=[concern, 'Time_Bin']).to_frame(index=False)

    all_combinations = pd.merge(all_combinations, grouped_data, on=[concern, 'Time_Bin'], how='left').fillna(0)


    # Create a figure with go.Figure
    fig = go.Figure()

    # Add traces for each agency
    for agence in top_agences:
        agency_data = all_combinations[all_combinations[concern] == agence]
        fig.add_trace(go.Scatter(
            x=agency_data['Time_Bin'],
            y=agency_data[time],
            mode='lines+markers',
            fill='tozeroy',
            name=agence
        ))

    # Update layout for better visualization
    fig.update_layout(
        title=title,
        xaxis_title=f'Intervalle de Temps en {unit}',
        yaxis_title='Temp Moyen (minutes)',
        template='plotly_dark',
        legend_title=concern
        
    )
    # Ajouter une ligne horizontale avec une couleur différente des courbes
    fig.add_shape(
        type="line",
        x0=all_combinations['Time_Bin'].min(),  # Début de la ligne sur l'axe x
        x1=all_combinations['Time_Bin'].max(),  # Fin de la ligne sur l'axe x
        y0=seuil,  # Position de la ligne sur l'axe y
        y1=seuil,  # Même que y0 pour que la ligne soit horizontale
        line=dict(color="yellow", width=2, dash="dot")  # Couleur différente (ici, noir)
    )
    
    # Display the chart in Streamlit
    return fig


In [172]:
area_graph(df,concern='NomAgence',time='TempOperation',date_to_bin='Date_Fin',seuil=5,title='Courbe')

In [154]:
all_combinations['NomAgence'].unique()

array(['Agence_1', 'Agence_19', 'Agence_16', 'Agence_13', 'Agence_6'],
      dtype=object)

In [114]:
import altair as alt
import pandas as pd
import streamlit as st

# Sample data
data = pd.DataFrame({
    'category': ['A', 'A', 'B', 'B', 'C', 'C'],
    'sub_group': ['X', 'Y', 'X', 'Y', 'X', 'Y'],
    'value': [10, 15, 20, 25, 30, 35]
})

# Create the Stacked Bar Chart
chart = alt.Chart(data).mark_bar().encode(
    x=alt.X('category:O', title='Category'),
    y=alt.Y('sum(value):Q', title='Value'),
    color=alt.Color('sub_group:N', title='Sub Group'),
    order=alt.Order('sub_group:N')  # To ensure stacking order
).properties(
    width=600,
    height=400
)

# Display in Streamlit
st.write("Stacked Bar Chart Example")
chart


alt.Chart(...)

In [71]:
df['Time_Bin'] = df['Date_Reservation'].apply(lambda x: assign_to_bin(x,bins))

In [77]:
df

,FirstName,LastName,UserName,Date_Reservation,Date_Appel,TempAttenteMoyen,TempsAttenteReel,Date_Fin,TempOperation,Nom,NomService,Type_Operation,AgenceId,NomAgence,Capacites,Longitude,Latitude,Time_Bin
0,LEON MARCELIN,NDONG,LNDONG2,2024-08-06 15:31:47.483,2024-08-06 17:51:14.157,38.0,8367.0,NaT,NaN,En attente,Caisse,None,32,Agence Principale,30,-17.469949,14.70138,2024-08-02 to 2024-08-06
1,LEON MARCELIN,NDONG,LNDONG2,2024-08-06 15:31:26.183,2024-08-06 17:21:41.437,37.0,6615.0,2024-08-06 17:51:13.153,1772.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138,2024-08-02 to 2024-08-06
2,LEON MARCELIN,NDONG,LNDONG2,2024-08-06 15:31:04.417,2024-08-06 17:20:47.487,37.0,6583.0,2024-08-06 17:21:41.433,54.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138,2024-08-02 to 2024-08-06
3,LEON MARCELIN,NDONG,LNDONG2,2024-08-06 15:30:42.700,2024-08-06 17:17:01.717,33.0,6379.0,2024-08-06 17:20:47.480,226.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138,2024-08-02 to 2024-08-06
4,LEON MARCELIN,NDONG,LNDONG2,2024-08-06 15:30:20.703,2024-08-06 17:08:04.010,33.0,5864.0,2024-08-06 17:21:40.533,816.0,Passée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138,2024-08-02 to 2024-08-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10079,LEON MARCELIN,NDONG,LNDONG2,2024-06-07 07:32:27.427,2024-06-07 08:05:09.973,0.0,1962.0,2024-06-07 08:08:19.713,190.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138,2024-06-07 to 2024-06-13
10080,AMINATA,SANE,amisane,2024-06-07 07:32:15.263,2024-06-07 07:59:42.827,0.0,1647.0,2024-06-07 08:13:43.730,841.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138,2024-06-07 to 2024-06-13
10081,AMINATA,SANE,amisane,2024-06-07 07:32:07.150,2024-06-07 07:50:59.710,0.0,1132.0,2024-06-07 07:59:42.820,523.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138,2024-06-07 to 2024-06-13
10082,AMINATA,SANE,amisane,2024-06-07 07:31:42.620,2024-06-07 07:50:48.230,0.0,1146.0,2024-06-07 07:50:59.703,11.0,Traitée,Caisse,None,32,Agence Principale,30,-17.469949,14.70138,2024-06-07 to 2024-06-13


[]